## **Code playground for SDA sem 12**


# Shortest path in a Graph


## Dijkstra's algorithm


Lets consider the following weighted undirected graph.


In [ ]:
undirected_graph = {
    0: [(2, 7), (1, 1)],
    1: [(0, 1), (2, 8), (3, 7), (4, 2)],
    2: [(0, 7), (1, 8), (4, 3)],
    3: [(1, 7), (4, 6)],
    4: [(3, 6), (1, 2), (2, 3)],
}

![Dijkstra undirected graph example](media/dijkstra_undirected_graph.png)


Dijksta's algorithm can find the shortest distance from a given source vertex to all others:


In [ ]:
from heapq import heappop, heappush

INF = float("infinity")


def dijkstra(start, V, graph):
    distances = [INF] * V
    distances[start] = 0

    visited = set()

    pq = [(0, start)]

    while pq:
        total_weight, current = heappop(pq)

        if current in visited:
            continue
        visited.add(current)

        for neighb, added_weight in graph[current]:
            if neighb in visited:
                continue

            new_weight = total_weight + added_weight

            if distances[neighb] == INF or new_weight < distances[neighb]:
                distances[neighb] = new_weight
                heappush(pq, (new_weight, neighb))

    return distances


start = 0
V = 5
distances = dijkstra(start, V, undirected_graph)
print(distances)  # [0, 1, 6, 8, 3]

[0, 1, 6, 8, 3]


Verbose version:


In [ ]:
from heapq import heappop, heappush

INF = float("infinity")


def dijkstra_verbose(start, V, graph):
    visited = set()

    distances = [INF] * V
    distances[start] = 0

    pq = [(0, start)]

    while pq:
        total_weight, current = heappop(pq)

        if current in visited:
            print(f"Node {current} has already been checked")
            continue
        visited.add(current)

        print(distances, f"Node: {current}, distance = {total_weight}")

        for neighb, added_weight in graph[current]:
            if neighb in visited:
                continue

            new_weight = total_weight + added_weight

            print(f"Check new distance to {neighb}: {new_weight} <? {distances[neighb]}")
            if distances[neighb] == INF or new_weight < distances[neighb]:
                distances[neighb] = new_weight
                heappush(pq, (new_weight, neighb))

        print("Priority Queue:", pq)
        print()

    return distances


start = 0
V = 5
distances = dijkstra_verbose(start, V, undirected_graph)
print(distances)  # [0, 1, 6, 8, 3]

[0, inf, inf, inf, inf] Node: 0, distance = 0
Check new distance to 2: 7 <? inf
Check new distance to 1: 1 <? inf
Priority Queue: [(1, 1), (7, 2)]

[0, 1, 7, inf, inf] Node: 1, distance = 1
Check new distance to 2: 9 <? 7
Check new distance to 3: 8 <? inf
Check new distance to 4: 3 <? inf
Priority Queue: [(3, 4), (8, 3), (7, 2)]

[0, 1, 7, 8, 3] Node: 4, distance = 3
Check new distance to 3: 9 <? 8
Check new distance to 2: 6 <? 7
Priority Queue: [(6, 2), (8, 3), (7, 2)]

[0, 1, 6, 8, 3] Node: 2, distance = 6
Priority Queue: [(7, 2), (8, 3)]

Node 2 has already been checked
[0, 1, 6, 8, 3] Node: 3, distance = 8
Priority Queue: []

[0, 1, 6, 8, 3]


Note that after the third iteration the priority queue has the _vertex 2_ two times. After the first _2_ is popped from the priority queue, the optimal path to _vertex 2_ was found. This means that each time we see the _vertex 2_ being popped from the priority queue, the iteration can be skipped. Also there is no need to calculate a potential shorter path to an already visited vertex. That is where the visited array comes into play.


Dijkstra does not work for graph having a negative weight edge among positive ones:


In [4]:
graph_negative_edge = {
    0: [(1, 1), (2, 7), (3, 5)],
    1: [(3, 1)],
    2: [(1, -10)],
    3: [],
}

![Dijkstra negative edge graph example](media/dijkstra_negative_edge_graph.png)


The shortest path to _vertex 1_ shall be _-3_, not _1_.
Also the shortest path to _vertex 3_ shall be _7 + (-10) + 1 = -2_. This is because _vertex 1_ is popped from the priority queue first and no alternative paths are ever considered after that.


In [ ]:
start = 0
V = 4
dijkstra(start, V, graph_negative_edge)  # [0, 1, 7, 2]

[0, 1, 7, 2]

Dijkstra can be modified so that a vertex may enter the priority queue multiple times - removing the _visited set_. The new algorithm will work with negative edges, but this is a controversial topic if it is still Dijkstra's algorithm and not something similar to Bellman-Ford's.


In [ ]:
from heapq import heappop, heappush

INF = float("infinity")


def dijkstra_modified(start, V, graph):
    distances = [INF] * V
    distances[start] = 0

    pq = [(0, start)]

    while pq:
        total_weight, current = heappop(pq)

        for neighb, added_weight in graph[current]:
            new_weight = total_weight + added_weight

            if distances[neighb] == INF or new_weight < distances[neighb]:
                distances[neighb] = new_weight
                heappush(pq, (new_weight, neighb))

    return distances


start = 0
V = 4
dijkstra_modified(start, V, graph_negative_edge)  # [0, -3, 7, -2]

[0, -3, 7, -2]

## Bellman-Ford algorithm


Lets search for the shortest path in the same graph from above:


In [7]:
graph_negative_edge = {
    0: [(1, 1), (2, 7), (3, 5)],
    1: [(3, 1)],
    2: [(1, -10)],
    3: [],
}

![Dijkstra negative edge graph example](media/dijkstra_negative_edge_graph.png)


In [ ]:
INF = float("infinity")


def bellman_ford(start, V, graph):
    distances = [INF] * V
    distances[start] = 0

    for _ in range(V - 1):
        for v in range(V):
            for child, weight in graph[v]:
                distances[child] = min(distances[child], distances[v] + weight)

    return distances


start = 0
V = 4
bellman_ford(start, V, graph_negative_edge)  # [0, -3, 7, -2]

[0, -3, 7, -2]

Note that the graph can be represented using list of edges:


In [ ]:
graph_list_of_edges = [
    (0, 1, 1),
    (0, 2, 7),
    (0, 3, 5),
    (1, 3, 1),
    (2, 1, -10),
]

In [ ]:
INF = float("infinity")


def bellman_ford_edges(start, V, graph):
    distances = [INF] * V
    distances[start] = 0

    for _ in range(V - 1):
        for x, y, w in graph:  # O(E)
            distances[y] = min(distances[y], distances[x] + w)

    return distances


start = 0
V = 4
bellman_ford_edges(start, V, graph_list_of_edges)  # [0, -3, 7, -2]

[0, -3, 7, -2]

Verbose version:


In [ ]:
INF = float("infinity")


def bellman_ford_verbose(start, V, graph):
    distances = [INF] * V
    distances[start] = 0

    for _ in range(V - 1):
        print(distances)

        for x, y, w in graph:  # O(E)
            print(x, "to", y, "dist", w)
            print(f"Check {distances[x]} + {w} <? {distances[y]}")
            distances[y] = min(distances[y], distances[x] + w)
        print()
    return distances


start = 0
V = 4
bellman_ford_verbose(start, V, graph_list_of_edges)  # [0, -3, 7, -2]

[0, inf, inf, inf]
0 to 1 dist 1
Check 0 + 1 <? inf
0 to 2 dist 7
Check 0 + 7 <? inf
0 to 3 dist 5
Check 0 + 5 <? inf
1 to 3 dist 1
Check 1 + 1 <? 5
2 to 1 dist -10
Check 7 + -10 <? 1

[0, -3, 7, 2]
0 to 1 dist 1
Check 0 + 1 <? -3
0 to 2 dist 7
Check 0 + 7 <? 7
0 to 3 dist 5
Check 0 + 5 <? 2
1 to 3 dist 1
Check -3 + 1 <? 2
2 to 1 dist -10
Check 7 + -10 <? -3

[0, -3, 7, -2]
0 to 1 dist 1
Check 0 + 1 <? -3
0 to 2 dist 7
Check 0 + 7 <? 7
0 to 3 dist 5
Check 0 + 5 <? -2
1 to 3 dist 1
Check -3 + 1 <? -2
2 to 1 dist -10
Check 7 + -10 <? -3



[0, -3, 7, -2]

Add the check if the graph has negative weight cycle:


In [ ]:
INF = float("infinity")


def bellman_ford_check(start, V, graph):
    distances = [INF] * V
    distances[start] = 0

    for _ in range(V - 1):
        for x, y, w in graph:  # O(E)
            distances[y] = min(distances[y], distances[x] + w)

    for x, y, w in graph:
        if distances[x] + w < distances[y]:
            raise Exception("Graph has a negative cycle")

    return distances


start = 0
V = 4
bellman_ford_check(start, V, graph_list_of_edges)  # [0, -3, 7, -2]

[0, -3, 7, -2]

For a graph with negative cycle:


In [ ]:
graph_negative_cycle = [
    (1, 0, 1),
    (0, 2, 7),
    (0, 3, 5),
    (1, 3, 1),
    (2, 1, -10),
]

In [16]:
start = 0
V = 4

try:
    bellman_ford_check(start, V, graph_negative_cycle)
except Exception as e:
    print("Caught Exception: " + str(e))

Caught Exception: Graph has a negative cycle
